<a href="https://colab.research.google.com/github/LuisPinheiro2611/NDVI/blob/main/ndvi_engine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import ee
import geopandas as gpd
import geemap

# Autenticação
ee.Authenticate()

#Inicializar
ee.Initialize(project='ndvi-457802')

#Abrir arquivo
area=gpd.read_file('area.shp')

crs_destino='EPSG:4326'

#Verificar a crs do arquivo
print(area.crs)

#Modificar crs
area=area.to_crs(crs_destino)

print(area.crs)

#Visualizar a área
area.boundary.plot()

#Transformar em .JSON para abrir no engine
area_json=area.boundary.to_json()

#Inicializar o projeto do google cloud
ee.Initialize(project='ndvi-457802')

#Converter arquivo json para o formato do google earth
aoi=ee.Geometry.Polygon(area.geometry[0].__geo_interface__['coordinates'])

#Coletar imagens do sentinel-2A
imagem=ee.ImageCollection('COPERNICUS/S2_SR').filterBounds(aoi).filterDate('2025-04-24','2025-04-27').sort('CLOUDY_COVER',False).first()

#Gerar NDVI
ndvi=imagem.normalizedDifference(['B8','B4']).rename('NDVI')
#Cortar NDVI de acordo com a área
ndvi_clip=ndvi.clip(aoi)

#Visualizar
ndvi_vis={'min':0.0,'max':1.0,'palette': ['red', 'orange', 'yellow', 'green']}

#Mapa
Map=geemap.Map()
Map.addLayer(ndvi_clip,ndvi_vis,'NDVI')
Map.centerObject(aoi,12)
Map.addLayer(aoi, {'color': 'red'}, 'Área de Interesse')

#Gerar barra de legenda
Map.add_colorbar(vis_params=ndvi_vis,label='NDVI',layer_name='NDVI')
Map



EPSG:31982
EPSG:4326


/usr/local/lib/python3.11/dist-packages/ee/deprecation.py:207: DeprecationWarning: 

Attention required for COPERNICUS/S2_SR! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

  warnings.warn(warning, category=DeprecationWarning)


Map(center=[-16.597587116092292, -49.29259188605676], controls=(WidgetControl(options=['position', 'transparen…

In [ ]:
#Exportar imagem NDVI
rask=geemap.ee_export_image(ndvi_clip,filename='ndvi.tif',scale=10)
